__[nvidia cuda python learning](https://github.com/apowers313/roc/blob/master/experiments/2024.09.02-17.58.36-nvidia-cuda-python-learning/2024.09.02-17.58.36-nvidia-cuda-python-learning.ipynb)__

In [ ]:
!date

In [ ]:
# save notebook path before we get started
import os

notebook_path = os.path.abspath("") # not sure if this or os.getcwd() is more reliable

In [ ]:
from baracuda import CudaDevice

print("Device Count:", CudaDevice.count())

dev = CudaDevice(0)
print("Device Name:", dev.name)
print("Compute Capability:", dev.compute_capability)
print("Driver Version:", dev.driver_version)

In [ ]:
from baracuda import CudaSourceFile

mod = CudaSourceFile("test_kernel.cu")
mod.call("test_kernel")

In [1]:
from baracuda import CudaSourceFile, CudaGraph

mod = CudaSourceFile("daisy_chain.cu")
g = CudaGraph()
k1 = mod.get_function("k1")
k2 = mod.get_function("k2")
g.add_kernel_node(k1)
g.add_kernel_node(k2)
nodes = g.nodes
print(f"Num of nodes in the graph created manually = {len(nodes)}")
g.run()

getting default context
creating context
CODE:
-------
extern "C" {
__global__ void k1() {
  printf("kernel 1 starting...\n");
  // cudaEvent_t e;
  // cudaEventCreateWithFlags(&e, cudaEventDisableTiming);
  printf("kernel 1 done.\n");
}

__global__ void k2() {
  printf("kernel 2 starting...\n");
  printf("kernel 2 done.\n");
}

}

-------

Compilation results 
getting default context
getting default context
Num of nodes in the graph created manually = 0


kernel 2 starting...
kernel 1 starting...
kernel 2 done.
kernel 1 done.
